In [ ]:
# Librería estándar
import argparse
import os
import warnings
import glob
# Librerías de terceros
import joblib
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    hamming_loss,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sagemaker import Session, image_uris


In [ ]:
def read_data(file_path):
    """
    Lee un archivo parquet y lo carga en un DataFrame de pandas.

    Parámetros
    ----------
    file_path: str
        Ruta al archivo parquet que se desea leer.

    Retorna
    -------
    pandas.DataFrame
        DataFrame con los datos cargados desde el archivo parquet.
    """
    data = pd.read_parquet(file_path)
    return data

In [ ]:
def col_vectorizer_tfidf(df, col):
    """
    Convierte la columna especificada a string, la vectoriza usando TF-IDF y retorna la matriz y el vectorizador.

    Parámetros
    ----------
    df: pandas.DataFrame
        DataFrame que contiene la columna a vectorizar.
    col: str
        Nombre de la columna de texto.

    Retorna
    -------
    X : scipy.sparse.csr.csr_matrix
        Matriz TF-IDF.
    vectorizer : TfidfVectorizer
        Vectorizador entrenado.
    """
    df[col] = df[col].astype(str)
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X = vectorizer.fit_transform(df[col].str.lower())
    return X, vectorizer


In [ ]:
def print_score(y_pred, clf):
    """
    Imprime métricas de evaluación de clasificación multilabel para un clasificador dado.

    Parámetros
    ----------
    y_pred : array-like
        Predicciones generadas por el clasificador.
    clf : objeto clasificador
        Instancia del modelo utilizado para predecir, se usa solo para mostrar su nombre de clase.

    Comportamiento
    -------------
    - Muestra por consola:
      * Nombre de la clase del clasificador.
      * Exactitud (accuracy).
      * Recall ponderado.
      * Precisión ponderada.
      * Puntuación F1 ponderada.
      * Puntuación de Jaccard promedio (porcentaje).
      * Hamming loss en porcentaje.
    - Asume que la variable global `y_test` contiene las etiquetas verdaderas.

    Retorna
    -------
    None
    """
    print("Clf: ", clf.__class__.__name__)
    print("Accuracy score: {}".format(accuracy_score(y_test, y_pred)))
    print("Recall score: {}".format(recall_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("Precision score: {}".format(precision_score(y_true=y_test, y_pred=y_pred, average='weighted')))
    print("F1 score: {}".format(f1_score(y_pred, y_test, average='weighted')))
    print("Jacard score: {}".format(avg_jacard(y_test, y_pred)))
    print("Hamming loss: {}".format(hamming_loss(y_pred, y_test)*100))
    print("---")


In [ ]:
def predict_top_n_tags(text, vectorizer, classifier, multilabel_binarizer, n=5):
    """
    Predice las n etiquetas más probables para un texto dado.

    Parámetros:
    -----------
    text : str
        Texto para el cual predecir etiquetas.
    vectorizer : TfidfVectorizer
        Vectorizador TF-IDF entrenado.
    classifier : OneVsRestClassifier
        Clasificador entrenado.
    multilabel_binarizer : MultiLabelBinarizer
        Binarizador de etiquetas utilizado.
    n : int
        Número máximo de etiquetas a predecir.

    Retorna:
    --------
    list
        Lista de las n etiquetas más probables.
    """
    # Transformar el texto usando el vectorizador
    X = vectorizer.transform([text.lower()])

    # Obtener probabilidades para cada clase
    y_proba = classifier.predict_proba(X)

    # Obtener los índices de las n probabilidades más altas
    top_n_indices = y_proba[0].argsort()[-n:][::-1]

    # Crear matriz binaria para las etiquetas top
    y_pred = np.zeros(y_proba.shape[1], dtype=int)
    for idx in top_n_indices:
        # Solo incluir si la probabilidad supera un umbral mínimo (opcional)
        if y_proba[0][idx] > 0.05:
            y_pred[idx] = 1

    # Convertir de matriz binaria a etiquetas
    predicted_tags = multilabel_binarizer.inverse_transform(y_pred.reshape(1, -1))

    return predicted_tags[0]


In [ ]:
data = pd.read_parquet("s3://prueba-tecnica-nequi-camilo/data/train/")

In [ ]:
data

In [ ]:
data['text'] = data['Title'] + " " + data['Body']

In [ ]:
X, vectorizer = col_vectorizer_tfidf(data, 'text')

In [ ]:
y = data['Tags']

In [ ]:
multilabel_binarizer = MultiLabelBinarizer()

In [ ]:
new_y = multilabel_binarizer.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, new_y)

In [ ]:
pipeline = Pipeline([
    ("clf", OneVsRestClassifier(
        LogisticRegression(max_iter=300)
    ))
])

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
data

In [ ]:
clf = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
clf.fit(X_train, y_train)

In [ ]:
import tarfile
joblib.dump(clf, "../../../Downloads/model.joblib")
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("model.joblib")

In [ ]:
from sagemaker import Session
sess = Session()
model_s3 = sess.upload_data("model.tar.gz", 
                            bucket="prueba-tecnica-nequi-camilo",
                            key_prefix="model-artifacts")

In [ ]:
from sagemaker.sklearn.model import SKLearnModel
from sagemaker import Session
from sagemaker import get_execution_role

sess = Session()
role = get_execution_role()
prefix = "model-artifacts"
model_artifact = f"s3://prueba-tecnica-nequi-camilo/{prefix}/model.tar.gz"

sklearn_model = SKLearnModel(
    model_data=model_artifact,
    role=role,
    framework_version="0.23-1",
    py_version="py3",
    sagemaker_session=sess
)

In [ ]:
model_package_arn = sklearn_model.register(
    model_package_group_name="PruebaTecnica",
    content_types=["tapplication/x-parquet"],
    response_types=["application/json"],
    inference_instances=["ml.m5.large"],
    transform_instances=["ml.m5.large"]
)
print("Registered ModelPackage ARN:", model_package_arn)

In [ ]:
predictor = sklearn_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
 )
